In [1]:
from pypekit import Task
import pandas as pd


class IrisLoader(Task):
    input_types = ["source"]
    output_types = ["raw"]

    def run(self, _):
        from sklearn.datasets import load_iris
        iris = load_iris()
        iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
        iris_df['target'] = iris.target
        return iris_df


class TrainTestSplitter(Task):
    input_types = ["raw"]
    output_types = ["split"]

    def run(self, df):
        from sklearn.model_selection import KFold
        kf = KFold(n_splits=self.run_config['n_splits'], shuffle=True, random_state=self.run_config['random_state'])
        split_index = self.run_config['split_index']
        train_indices, test_indices = list(kf.split(df))[split_index]

        train_df = df.iloc[train_indices].reset_index(drop=True)
        test_df = df.iloc[test_indices].reset_index(drop=True)

        train_df['train'] = 1
        test_df['train'] = 0
        df = pd.concat([train_df, test_df], ignore_index=True)
        return df


class Scaler(Task):
    input_types = ["split"]
    output_types = ["processed"]

    def run(self, df):
        X = df.drop(columns=['target', 'train'])
        X_train = X[df['train'] == 1]

        scaler = self.get_scaler()
        scaler.fit(X_train)

        X_scaled = scaler.transform(X)
        scaled_df = pd.DataFrame(data=X_scaled, columns=X.columns)
        scaled_df['target'] = df['target']
        scaled_df['train'] = df['train']

        return scaled_df

    def get_scaler(self):
        raise NotImplementedError("Subclasses should implement this method.")


class MinMaxScaler(Scaler):
    def get_scaler(self):
        from sklearn.preprocessing import MinMaxScaler
        return MinMaxScaler()


class StandardScaler(Scaler):
    def get_scaler(self):
        from sklearn.preprocessing import StandardScaler
        return StandardScaler()


class PCA(Task):
    input_types = ["processed"]
    output_types = ["pca"]

    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def run(self, df):
        X = df.drop(columns=['target', 'train'])
        X_train = X[df['train'] == 1]

        from sklearn.decomposition import PCA
        pca = PCA(**self.kwargs)
        pca.fit(X_train)

        X_pca = pca.transform(X)
        pca_df = pd.DataFrame(data=X_pca, columns=[
                              f'PC{i+1}' for i in range(X_pca.shape[1])])
        pca_df['target'] = df['target']
        pca_df['train'] = df['train']

        return pca_df


class Classifier(Task):
    input_types = ["split", "processed", "pca"]
    output_types = ["predicted"]

    def run(self, df):
        X = df.drop(columns=['target', 'train'])
        y = df['target']
        X_train = X[df['train'] == 1]
        y_train = y[df['train'] == 1]

        classifier = self.get_classifier()
        classifier.fit(X_train, y_train)
        
        y_pred = classifier.predict(X)
        df['predicted'] = y_pred

        return df
    
    def get_scaler(self):
        raise NotImplementedError("Subclasses should implement this method.")


class LogisticRegression(Classifier):
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def get_classifier(self):
        from sklearn.linear_model import LogisticRegression
        return LogisticRegression(**self.kwargs)


class RandomForestClassifier(Classifier):
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def get_classifier(self):
        from sklearn.ensemble import RandomForestClassifier
        return RandomForestClassifier(**self.kwargs)


class SVC(Classifier):
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def get_classifier(self):
        from sklearn.svm import SVC
        return SVC(**self.kwargs)


class Evaluator(Task):
    input_types = ["predicted"]
    output_types = ["sink"]

    def run(self, df):
        df_test = df[df['train'] == 0]
        return (df_test['target'] == df_test['predicted']).mean()

In [2]:
from pypekit import Repository, CachedExecutor

repository = Repository([
    IrisLoader,
    TrainTestSplitter,
    MinMaxScaler,
    StandardScaler,
    PCA,
    LogisticRegression,
    RandomForestClassifier,
    SVC,
    Evaluator
])

repository.build_tree()
print(repository.build_tree_string())

└── Root()
    └── IrisLoader()
        └── TrainTestSplitter()
            ├── MinMaxScaler()
            │   ├── PCA()
            │   │   ├── LogisticRegression()
            │   │   │   └── Evaluator()
            │   │   ├── RandomForestClassifier()
            │   │   │   └── Evaluator()
            │   │   └── SVC()
            │   │       └── Evaluator()
            │   ├── LogisticRegression()
            │   │   └── Evaluator()
            │   ├── RandomForestClassifier()
            │   │   └── Evaluator()
            │   └── SVC()
            │       └── Evaluator()
            ├── StandardScaler()
            │   ├── PCA()
            │   │   ├── LogisticRegression()
            │   │   │   └── Evaluator()
            │   │   ├── RandomForestClassifier()
            │   │   │   └── Evaluator()
            │   │   └── SVC()
            │   │       └── Evaluator()
            │   ├── LogisticRegression()
            │   │   └── Evaluator()
            │   ├── RandomForestCla

In [3]:
pipelines = repository.build_pipelines()
pipelines

[Pipeline(tasks=[IrisLoader(), TrainTestSplitter(), MinMaxScaler(), PCA(), LogisticRegression(), Evaluator()]),
 Pipeline(tasks=[IrisLoader(), TrainTestSplitter(), MinMaxScaler(), PCA(), RandomForestClassifier(), Evaluator()]),
 Pipeline(tasks=[IrisLoader(), TrainTestSplitter(), MinMaxScaler(), PCA(), SVC(), Evaluator()]),
 Pipeline(tasks=[IrisLoader(), TrainTestSplitter(), MinMaxScaler(), LogisticRegression(), Evaluator()]),
 Pipeline(tasks=[IrisLoader(), TrainTestSplitter(), MinMaxScaler(), RandomForestClassifier(), Evaluator()]),
 Pipeline(tasks=[IrisLoader(), TrainTestSplitter(), MinMaxScaler(), SVC(), Evaluator()]),
 Pipeline(tasks=[IrisLoader(), TrainTestSplitter(), StandardScaler(), PCA(), LogisticRegression(), Evaluator()]),
 Pipeline(tasks=[IrisLoader(), TrainTestSplitter(), StandardScaler(), PCA(), RandomForestClassifier(), Evaluator()]),
 Pipeline(tasks=[IrisLoader(), TrainTestSplitter(), StandardScaler(), PCA(), SVC(), Evaluator()]),
 Pipeline(tasks=[IrisLoader(), TrainTest

In [4]:
executor = CachedExecutor(pipelines, verbose=True)

n_splits = 5
results_dfs = []
for i in range(n_splits):
    results = executor.run(run_config={"n_splits": n_splits, "split_index": i, "random_state": 42})
    results = pd.DataFrame(results).transpose()
    results.reset_index(inplace=True, names="pipeline")
    results_dfs.append(results)

Pipeline 1/15 completed. Runtime: 1.12s.
Pipeline 2/15 completed. Runtime: 1.25s.
Pipeline 3/15 completed. Runtime: 1.09s.
Pipeline 4/15 completed. Runtime: 1.01s.
Pipeline 5/15 completed. Runtime: 1.08s.
Pipeline 6/15 completed. Runtime: 1.01s.
Pipeline 7/15 completed. Runtime: 1.01s.
Pipeline 8/15 completed. Runtime: 1.08s.
Pipeline 9/15 completed. Runtime: 1.01s.
Pipeline 10/15 completed. Runtime: 1.01s.
Pipeline 11/15 completed. Runtime: 1.08s.
Pipeline 12/15 completed. Runtime: 1.01s.
Pipeline 13/15 completed. Runtime: 1.01s.
Pipeline 14/15 completed. Runtime: 1.08s.
Pipeline 15/15 completed. Runtime: 1.00s.
Pipeline 1/15 completed. Runtime: 0.01s.
Pipeline 2/15 completed. Runtime: 0.07s.
Pipeline 3/15 completed. Runtime: 0.01s.
Pipeline 4/15 completed. Runtime: 0.01s.
Pipeline 5/15 completed. Runtime: 0.07s.
Pipeline 6/15 completed. Runtime: 0.01s.
Pipeline 7/15 completed. Runtime: 0.01s.
Pipeline 8/15 completed. Runtime: 0.08s.
Pipeline 9/15 completed. Runtime: 0.01s.
Pipeline 1

In [5]:
result_df = pd.concat(results_dfs)
accuracies_df = result_df.groupby("pipeline").agg(
    tasks=("tasks", "first"),
    mean_runtime=("runtime", "mean"),
    mean_accuracy=("output", "mean"),
)
accuracies_df.sort_values("mean_accuracy", ascending=False, inplace=True)
accuracies_df

,tasks,mean_runtime,mean_accuracy
pipeline,,,
3182421f14d64f3397088c7a634916bf,"[IrisLoader(), TrainTestSplitter(), LogisticRe...",0.208934,0.973333
0a1a56a9a5aa4e8298c3a6d2cd9847e5,"[IrisLoader(), TrainTestSplitter(), MinMaxScal...",0.206554,0.966667
0f11a79a8ee44f7aa36b4aacda8afc72,"[IrisLoader(), TrainTestSplitter(), MinMaxScal...",0.277563,0.966667
17ab0fac5d7246c5a7b72eeeeb3c1991,"[IrisLoader(), TrainTestSplitter(), SVC(), Eva...",0.204359,0.966667
40fc05e671d6417e81b2f67f2b34545e,"[IrisLoader(), TrainTestSplitter(), RandomFore...",0.2757,0.966667
0dea03d66bd942f3ab32f699840d678f,"[IrisLoader(), TrainTestSplitter(), MinMaxScal...",0.224491,0.953333
c9435710e8bf4c7cb92ccd934afc92f7,"[IrisLoader(), TrainTestSplitter(), MinMaxScal...",0.313991,0.953333
0e3cc014e65644309f871ad1a859fb8d,"[IrisLoader(), TrainTestSplitter(), StandardSc...",0.206333,0.953333
2f03659ce3704eb38db980bb52c938b5,"[IrisLoader(), TrainTestSplitter(), StandardSc...",0.207168,0.953333
